# Display MISP deleted attributes 

In [26]:
import urllib3
from pymisp import PyMISP, MISPEvent
import sys
sys.path.insert(0, "/home/koenv/cti-operational-procedure/vault/")
from keys import misp_url, misp_key, misp_verifycert

if misp_verifycert is False:
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

misp = PyMISP(misp_url, misp_key, misp_verifycert)
    
print("I will use the server {}".format(misp_url))

import requests
import tabulate
misp_headers = { "Authorization": misp_key, "Content-Type": "application/json", "Accept": "application/json" }

The version of PyMISP recommended by the MISP instance (2.4.162) is newer than the one you're using now (2.4.159). Please upgrade PyMISP.


I will use the server https://misp.demo.cudeso.be/


## List of "to delete" attributes

The attribute restsearch allows to search for **deleted** attributes by setting `deleted` to true. Note that you have to do this by supplying the value **1**, not the value **0**.

In [27]:
data = { "returnFormat": "json", "deleted": "1" }

In [28]:
result = requests.post("{}/{}".format(misp_url, "attributes/restSearch"), headers=misp_headers, verify=misp_verifycert, json=data)
deleted_attributes = []
for attr in result.json()["response"]["Attribute"]:
    deleted_attributes.append(
            [ attr["event_id"], attr["Event"]["info"], attr["id"], attr["value"]]
    )

print(tabulate.tabulate(deleted_attributes, headers=["Event ID", "Event title", "Attribute ID", "Attribute value"]))

  Event ID  Event title            Attribute ID  Attribute value
----------  -------------------  --------------  -----------------
       111  Basic malware event         2278551  my_bad_file.exe
       965  Demo Event 2022             2300697  www.google.com


## Recover

Recover a deleted attribute by updating the **attribute_id** value with the desired attribute ID

In [ ]:
attribute_id = 0

In [ ]:
result = requests.post("{}/{}/{}".format(misp_url, "attributes/restore", attribute_id), headers=misp_headers, verify=misp_verifycert, json=data)
if "Attribute" in result.json():
    print("Attribute recovered")
else:
    print(result.json())

## Purge

Permanently delete an attribute by updating the **attribute_id** value with the desired attribute ID. This action **cannot** be undone.

In [ ]:
result = requests.post("{}/{}/{}/true".format(misp_url, "attributes/delete", attribute_id), headers=misp_headers, verify=misp_verifycert, json=data)
print(result.json())